## UFO $\to$ Datumaro 변환

Datumaro 형식 참고 : https://docs.cvat.ai/docs/manual/advanced/formats/format-datumaro/

In [15]:
# numpy 버전을 2 이하로 맞춰야 함. requirement에 numpy 1.24.4로 기재되어있으므로 이 버전으로 바꾸는 거 추천함
import numpy as np
print(np.__version__)

1.24.4


In [16]:
#!pip install numpy==1.24.4

In [17]:
#!pip install -qqq treescope==0.1.1

In [18]:
#!apt-get update
#!apt-get install -y zip

In [19]:
import json
from treescope import display as tdisp
from matplotlib import pyplot as plt
from utils import read_json, datum_aro_2_ufo_reduced
from IPython.display import Image as idisp

In [20]:
ufo_in = read_json("/data/ephemeral/home/kjh/level2-cv-datacentric-cv-16/data/vietnamese_receipt/ufo/erased_empty_transcription.json")

In [21]:
# 파일 관련 설정

TASK_NAME = "receipt_vietname_trans" # CVAT 페이지에서 사용할 task 이름

SAVE_DIR = 'annotations'  # 변경한 json을 저장할 디렉토리 이름
SAVE_DIR_BACKUP = 'annotations_backup'  # 변경한 json을 모아둘 디렉토리 이름
#압축 과정에서 annotations 폴더를 초기화 하기 때문에, 백업용 폴더를 따로 만들어둔 것임
ZIP_DIR = 'zip'

ZIP_FILENAME = 'annotations_madeup_vi_trans' # CVAT에 업로드하기 위한 ZIP 파일 이름

SAVE_FILENAME = 'vietnam_trans'  # 저장할 json 파일 이름

In [22]:
# 돈 터치
SAVE_PATH = f'{SAVE_DIR}/{SAVE_FILENAME}.json'
SAVE_PATH_B = f'{SAVE_DIR_BACKUP}/{SAVE_FILENAME}.json'

In [23]:
def split_polygon(points:list[tuple[int,int]])->list[list[tuple[int,int]]]:
    if len(points) <= 4:
        return [points]

    # 첫 번째 사각형: 왼쪽 영역
    rect1 = [
        points[0],  # 왼쪽 상단
        points[1],  # 중간 상단
        points[4],  # 중간 하단
        points[5]   # 왼쪽 하단
    ]

    # 두 번째 사각형: 오른쪽 영역
    rect2 = [
        points[1],  # 중간 상단
        points[2],  # 오른쪽 상단
        points[3],  # 오른쪽 하단
        points[4]   # 중간 하단
    ]

    return [rect1, rect2]

    raise NotImplementedError

In [24]:
task_name = TASK_NAME
split = "train"
id_prefix = task_name + "/images/" + split + "/"

wrap_point_items = lambda items: {
    "info": {},
    "categories": {
        "label": {
            "labels": [{"name": "1", "parent": "", "attributes": []}],
            "attributes": ["occluded"],
        },
        "points": {"items": []},
    },
    "items": items,
}

get_image_node = lambda img_name: ufo_in["images"][img_name]

node_dimensions = lambda img_node: {
    "img_w": img_node["img_w"],
    "img_h": img_node["img_h"],
}

def wrap_annotations(idx: int, img_name: str, annotations: list[dict]):
    return {
        "id": id_prefix + img_name,
        #"id": img_name,
        "annotations": annotations,
        "attr": {"frame": idx},
        "point_cloud": {"path": ""},
        "info": node_dimensions(get_image_node(img_name)),
    }

def wrap_vertices(vertices: list[float]):
    return {
        "id": 0,
        "type": "polygon",
        "attributes": {"occluded": False},
        "group": 0,
        "label_id": 0,
        "points": vertices,
        "z_order": 0,
    }

In [25]:
# Datumaro 포맷의 Annotation 객체 생성
from utils import extract_flat_points

# 이미지 파일 이름 추출 및 정렬
image_filenames = sorted([img_name for img_name in ufo_in["images"].keys()])

# 정렬된 순서로 처리
image_map = {k: ufo_in["images"][k]["words"] for k in image_filenames}
flat_points = {fname: extract_flat_points(image) for fname, image in image_map.items()}

# 정렬된 순서로 annotation 생성
annotation = wrap_point_items(
    [
        wrap_annotations(
            idx_, img_name, [wrap_vertices(vertices) for vertices in flat_points[img_name]]
        )
        for idx_, img_name in enumerate(image_filenames)
    ]
)

In [26]:
# datumaro json 파일을 저장할 디렉토리 생성, 추후 압축을 위해 비워준다
!rm -rf "./{SAVE_DIR}"
!mkdir -p "./{SAVE_DIR}"
!mkdir -p "./{SAVE_DIR_BACKUP}"
!mkdir -p "./{ZIP_DIR}"

In [27]:
# 위에서 생성한 Annotation 객체를 json 파일로 저장
with open(SAVE_PATH, 'w') as f:
    json.dump(annotation, f, indent=2, ensure_ascii=False)
with open(SAVE_PATH_B, 'w') as f:
    json.dump(annotation, f, indent=2, ensure_ascii=False)

In [28]:
# CVAT에서 쓸 수 있는 zip 파일 생성
!zip -r ./{ZIP_DIR}/{ZIP_FILENAME}.zip {SAVE_DIR}

  adding: annotations/ (stored 0%)
  adding: annotations/vietnam_trans.json (deflated 91%)
